<a href="https://colab.research.google.com/github/danielramo27s/UDEA-ai4eng-20242/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>